# Dijkstra rank

<b>Definition:</b> <br>
When running Dijkstra’s
algorithm from a vertex s, the rank of a vertex u is the order in which it is taken from
the priority queue. [1]

In [1]:
import networkx as nx
from heapq import heappush, heappop
from itertools import count

## Test and understand the binary heap queu 

In [2]:
h = []
heappush(h, (5,4, 'write code'))
heappush(h, (7,1, 'release product'))
heappush(h, (1,3, 'write spec'))
heappush(h, (3,10, 'create tests'))
heappop(h)

(1, 3, 'write spec')

## Build and test a Dijkstra search algo to keep the dijkstra rank

In [3]:
G = nx.Graph()
G.add_edge(1,2, **{'name':'first','weight':1})
G.add_edge(1,3,**{'name':'first','weight':1})
G.add_edge(1,9,**{'name':'first','weight':2.2})
G.add_edge(1,4,**{'name':'first','weight':3})
G.add_edge(1,5,**{'name':'first','weight':4})
G.add_edge(2,8,**{'name':'first','weight':1})
G.add_edge(3,6,**{'name':'first','weight':12})
G.add_edge(3,70,**{'name':'first','weight':21})
G.add_edge(70,80,**{'name':'first','weight':21})

In [4]:
def weight(u, v, data):
    return lambda u, v, data: data.get(weight_str, 1)

def dijkstra(
    G, source, weight_str, pred=None, paths=None, cutoff=None, target=None, _print=False
):

    G_succ = G._succ if G.is_directed() else G._adj
    
    weight = lambda u, v, data: data.get(weight_str, 1)
    
    push = heappush
    pop = heappop
    dist = {}  # dictionary of final distances
    seen = {}
    # fringe is heapq with 3-tuples (distance,c,node)
    # use the count c to avoid comparing nodes (may not be able to)
    c = count()
    fringe = []
    if source not in G:
        raise nx.NodeNotFound(f"Source {source} not in G")
        
    if target not in G:
        raise nx.NodeNotFound(f"Target {source} not in G")

    seen[source] = 0
    push(fringe, (0, next(c), source))
    while fringe:
        if _print:
            print(fringe)
        (d, rank, v) = pop(fringe)
        if v in dist:
            continue  # already searched this node.
            
        dist[v] = d
        if v == target:
            break
        for u, e in G_succ[v].items():
            cost = e.get(weight_str, 1)
            if cost is None:
                continue
            vu_dist = dist[v] + cost
            
            if cutoff is not None:
                if vu_dist > cutoff:
                    continue
            if u in dist:
                u_dist = dist[u]
                if vu_dist < u_dist:
                    raise ValueError("Contradictory paths found:", "negative weights?")
                elif pred is not None and vu_dist == u_dist:
                    pred[u].append(v)
            elif u not in seen or vu_dist < seen[u]:
                seen[u] = vu_dist
                push(fringe, (vu_dist, next(c), u))
                if paths is not None:
                    paths[u] = paths[v] + [u]
                if pred is not None:
                    pred[u] = [v]
            elif vu_dist == seen[u]:
                if pred is not None:
                    pred[u].append(v)

    # The optional predecessor and path dictionaries can be accessed
    # by the caller via the pred and paths objects passed as arguments.
    return dist, rank

In [6]:
target = 80
source = 1
paths = {source:[source]}
dist, rank = dijkstra(G, 
                      source=source, 
                      pred={}, 
                      weight_str='weight', 
                      paths=paths, 
                      cutoff=None, 
                      target=target,
                      _print=True)

[(0, 0, 1)]
[(1, 1, 2), (1, 2, 3), (2.2, 3, 9), (3, 4, 4), (4, 5, 5)]
[(1, 2, 3), (2, 6, 8), (2.2, 3, 9), (4, 5, 5), (3, 4, 4)]
[(2, 6, 8), (3, 4, 4), (2.2, 3, 9), (4, 5, 5), (13, 7, 6), (22, 8, 70)]
[(2.2, 3, 9), (3, 4, 4), (22, 8, 70), (4, 5, 5), (13, 7, 6)]
[(3, 4, 4), (4, 5, 5), (22, 8, 70), (13, 7, 6)]
[(4, 5, 5), (13, 7, 6), (22, 8, 70)]
[(13, 7, 6), (22, 8, 70)]
[(22, 8, 70)]
[(43, 9, 80)]


In [7]:
dist[target]

43

In [8]:
rank

9

In [9]:
paths[target]

[1, 3, 70, 80]

In [10]:
paths

{1: [1],
 2: [1, 2],
 3: [1, 3],
 9: [1, 9],
 4: [1, 4],
 5: [1, 5],
 8: [1, 2, 8],
 6: [1, 3, 6],
 70: [1, 3, 70],
 80: [1, 3, 70, 80]}

## Compute path searches on the multimodal graph 

In [11]:
import osmnx as ox

import sys
sys.path.append('../../../Multimodal_freight_USA/')
from mfreight.Multimodal.graph_utils import MultimodalNet

In [12]:
Net = MultimodalNet(path_u= "../../mfreight/Multimodal/data/multimodal_G_tot_u_w_price.plk")

# Medium haul

In [14]:
price_target = "('AR', 'CA')"
target = 'CO2_eq_kg'

In [15]:
def loop_dijkstra_rank(departures, arrivals, target):
    ranks = []
    paths_len = []
    for departure, arrival in zip(departures.values(), arrivals.values()):
        node_orig = ox.get_nearest_node(Net.G_reachable_nodes, departure, method="haversine", return_dist=False)
        node_dest = ox.get_nearest_node(Net.G_reachable_nodes, arrival, method="haversine", return_dist=False)
        paths = {node_orig:[node_orig]}
        _, rank = dijkstra(Net.G_multimodal_u, 
                              source=node_orig, 
                              pred={}, 
                              weight_str=target, 
                              paths=paths, 
                              cutoff=None, 
                              target=node_dest)
        ranks.append(rank)
        paths_len.append(len(paths[node_dest]))
        
    return ranks, paths_len

In [16]:
departures={'Chicago':(41.763705, -87.714903),
            'Denver':(39.665040, -105.046780),
            'Dallas':(32.772557, -96.774195)}

arrivals={'Atlanta':(33.654622, -84.249274),
          'Phoenix':(33.438505, -112.075594),
          'Louisville':(38.246108, -85.744561)}

distance = [723,
           817,
           836]

In [19]:
ranks, paths_len = loop_dijkstra_rank(departures=departures, arrivals=arrivals, target='CO2_eq_kg')
for r,p,d in zip(ranks, paths_len, distance):
    print(f'rank: {r}, path_weight: {p}, distance [miles]: {d}')

rank: 28565, path_weight: 174, distance [miles]: 723
rank: 16013, path_weight: 138, distance [miles]: 817
rank: 19082, path_weight: 121, distance [miles]: 836


In [20]:
ranks, paths_len = loop_dijkstra_rank(departures, arrivals, "('AR', 'CA')")
for r,p,d in zip(ranks, paths_len, distance):
    print(f'rank: {r}, path_weight: {p}, distance [miles]: {d}')

rank: 41689, path_weight: 174, distance [miles]: 723
rank: 21590, path_weight: 90, distance [miles]: 817
rank: 30253, path_weight: 121, distance [miles]: 836


All theses origin destiation are chosen not to be too close form the ocean (this would bias the rank)

# Long haul

In [21]:
departures={'LA':(33.998788, -118.226320),
            'Vega':(36.097687, -115.201823),
            'Seatle':(29.739678, -95.378221)}

arrivals={'Seatle':(47.612580, -122.312518),
          'Nashville':(36.155835, -86.797575),
          'Houston':(47.612580, -122.312518)}

distance = [1140,
           1793,
           2300]

In [22]:
ranks, paths_len = loop_dijkstra_rank(departures, arrivals, 'CO2_eq_kg')
for r,p,d in zip(ranks, paths_len, distance):
    print(f'rank: {r}, path_weight: {p}, distance [miles]: {d}')

rank: 16350, path_weight: 231, distance [miles]: 1140
rank: 46100, path_weight: 264, distance [miles]: 1793
rank: 95892, path_weight: 291, distance [miles]: 2300


In [23]:
import time
start=time.time()

ranks, paths_len = loop_dijkstra_rank(departures, arrivals, "('AR', 'CA')")
print(f'elapsed time {time.time()-start}')
for r,p,d in zip(ranks, paths_len, distance):
    print(f'rank: {r}, path_weight: {p}, distance [miles]: {d}')

elapsed time 3.2993290424346924
rank: 21485, path_weight: 238, distance [miles]: 1140
rank: 61884, path_weight: 224, distance [miles]: 1793
rank: 101674, path_weight: 291, distance [miles]: 2300


# Observations

The rank is proportionnal to the distance between the origin and the destination when the routes are not too close to the borders of hte graph. 
The rank varies depending on the target weight. 

## Ref

[1]Peter Sanders and Dominik Schultes. Highway hierarchies hasten exact shortest path queries. In Proceedings of the 13th Annual European Symposium on Algorithms (ESA’05), volume 3669 of Lecture Notes in Computer Science, pages 568–579. Springer, 2005.